# interpret static embedding

In [1]:
from pathlib import Path
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
import matplotlib as mpl
from tqdm import tqdm
import torch
from ruamel.yaml import YAML
mpl.rcParams['figure.dpi'] = 150

In [2]:
# data_dir = Path('data/')
data_dir = Path('/cats/datastore/data/')

assert data_dir.exists()

In [3]:
%load_ext autoreload
%autoreload 2

# Code

In [4]:
print([d.name for d in (data_dir / "runs/").iterdir()])
print([d.name for d in (data_dir / "runs/ealstm_less_vars_2004_1607_1334").glob("*")])

(data_dir / "runs/ealstm_less_vars_2004_1607_1334")
(data_dir / "runs/lstm_less_vars_2004_1507_1028")

['ealstm_less_vars_2004_1607_1334', 'lstm_less_vars_1307_1717', 'ealstm_less_vars_2004_1707_1424', 'lstm_less_vars_2004_1507_1028']
['config.yml', 'events.out.tfevents.1594906656.GPU_MachineLearning.22441.0', 'model_epoch001.pt', 'model_epoch002.pt', 'model_epoch003.pt', 'model_epoch004.pt', 'model_epoch005.pt', 'model_epoch006.pt', 'model_epoch007.pt', 'model_epoch008.pt', 'model_epoch009.pt', 'model_epoch010.pt', 'model_epoch011.pt', 'valid_ds.nc', 'results_ealstm_less_vars_2004_1607_1334_E002.csv', 'results_ealstm_less_vars_2004_1607_1334_E006.csv', 'model_epoch012.pt', 'results_ealstm_less_vars_2004_1607_1334_E007.csv', 'results_ealstm_less_vars_2004_1607_1334_E001.csv', 'results_ealstm_less_vars_2004_1607_1334_E008.csv', 'results_ealstm_less_vars_2004_1607_1334_E009.csv', 'results_ealstm_less_vars_2004_1607_1334_E010.csv', 'model_epoch013.pt', 'results_ealstm_less_vars_2004_1607_1334_E005.csv', 'results_ealstm_less_vars_2004_1607_1334_E004.csv', 'results_ealstm_less_vars_2004_1607

PosixPath('/cats/datastore/data/runs/lstm_less_vars_2004_1507_1028')

In [5]:
model_path = (data_dir / "runs/ealstm_less_vars_2004_1607_1334/model_epoch014.pt") 
config_path = (data_dir / "runs/ealstm_less_vars_2004_1607_1334/config.yml") 
yaml = YAML(typ="safe")
cfg = yaml.load(config_path)
model_dict = torch.load(model_path)

In [19]:
assert (data_dir / "CAMELS_GB_DATASET").exists()
assert (data_dir / "runs/lstm_less_vars_2004_1507_1028/train_data/train_data_scaler.p").exists()
assert (data_dir / "runs/lstm_less_vars_2004_1507_1028/train_data/train_data.h5").exists()

cfg["data_dir"] = data_dir / "CAMELS_GB_DATASET"
cfg["scaler_file"] = data_dir / "runs/lstm_less_vars_2004_1507_1028/train_data/train_data_scaler.p"
cfg["h5_file"] = data_dir / "runs/lstm_less_vars_2004_1507_1028/train_data/train_data.h5"

# input_size_dyn = len(cfg["dynamic_inputs"])
# input_size_stat = len(cfg["static_inputs"] + cfg["camels_attributes"])

In [20]:
import sys
sys.path.insert(1, '/home/tommy/tommy_multiple_forcing')

from codebase.modelzoo.ealstm import EALSTM
from codebase.modelzoo.cudalstm import CudaLSTM
from codebase.data.camelstxt import CamelsGBCSV
from codebase.data.utils import load_basin_file
from codebase.data.camelsh5 import CamelsGBH5

In [ ]:
input_size_dyn = len(cfg["dynamic_inputs"])
input_size_stat = len(cfg["static_inputs"] + cfg["camels_attributes"])

# Initialize model
model = EALSTM(cfg)
model.load_state_dict(torch.load(model_path, map_location="cpu"))

# # extract weight and bias of input gate
# weight = model.lstm.weight_sh
# bias = model.lstm.bias_s

# load in the attributes
h5 = CamelsGBH5(cfg)
attributes = h5._load_attributes()
# means = attributes.mean()
# stds = attributes.std()

In [23]:
basins = load_basin_file(cfg["train_basin_file"])
basin = basins[0]
ds_test = CamelsGBCSV(camels_root=cfg["data_dir"],
                        basin=basin,
                        dates=[cfg["test_start_date"], cfg["test_end_date"]],
                        is_train=False,
                        with_attributes=True,
                        attribute_means=means,
                        attribute_stds=stds,
                        db_path=str(BASE_RUN_DIR / "attributes.db"))

AttributeError: 'str' object has no attribute 'open'

In [ ]:
for basin in tqdm(basins):
    ds_test = CamelsTXT(camels_root=CAMELS_DIR,
                        basin=basin,
                        dates=[VAL_START, VAL_END],
                        is_train=False,
                        with_attributes=True,
                        attribute_means=means,
                        attribute_stds=stds,
                        db_path=str(BASE_RUN_DIR / "attributes.db"))
    input_gate = torch.sigmoid(torch.addmm(bias, ds_test.attributes, weight))
    lstm_embedding[basin] = input_gate.detach().numpy()
